## Simulation Framework 

In [4]:
# import functions from model_evaluation.ipynb 
import import_ipynb
import model_evaluation 

In [5]:
# Define input variables  
groups_1 = [[1, 2, 3], [1], [0, 2]] 
votes_1 = [5, 5, 5, 5]

In [6]:
# Assuming this is the correct function signature
result = model_evaluation.group_memberships(groups_1, votes_1)
print(result)

[[2], [0, 1], [0, 2], [0]]
